# Google Authentication

In [1]:
import sys
if 'google.colab' in sys.modules:
    !pip install langchain google-cloud-aiplatform rich
    google_auth.authenticate_user()

In [2]:

import os
import requests

In [3]:

from langchain.llms import VertexAI
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain

In [4]:
from google.cloud import aiplatform
#! Change the following to your own project and location
aiplatform.init(project="aerobic-gantry-387923", location="us-central1")
vertex = VertexAI()

In [5]:
import Tools
agent_tools = Tools.Tools()

tools = [
    Tool(
        name = "Degree Search",
        func = agent_tools.get_degrees_from_query,
        description = "This tool allows you to search for UT Dallas' Degrees, Majors, Minors, and Certificates using keywords. Use this tool to compare multiple degrees. It returns relevant URLs, titles, and snippets from the search engine results.",
    ),
    Tool(
        name = "Course Search",
        func = agent_tools.get_courses_from_query,
        description = "This tool enables you to search for UT Dallas' courses using keywords. Use this tool to compare multiple courses. It returns the course numbers/codes where the keyword exists in the course title.",
    ),

    Tool(
        name = "General Search",
        func = agent_tools.utdallas_search,
        description = "Please use this tool only as a last resort. It performs a comprehensive search across all UT Dallas pages that will not give information about courses or degrees. It requires a query string and returns possible links and snippet contents. Do not use for course search or degree search as there are separate tools for those.",
    ),
    Tool(
        name = "Extract Text from utdallas.edu site from Course, Degree, or Website Search Tool",
        func = agent_tools.access_utdallas_site,
        description = "If you have a URL string containing utdallas.edu, use this function to extract the text content from that page.",
    ),
    Tool(
        name = "Dictionary Search",
        func = agent_tools.dictionary_search,
        description = "If you need the definition of a word or phrase unrelated to UT Dallas, use this function to retrieve the definition. Do not use for anything related to UT Dallas' courses.",
    )
]


In [6]:
prefix = """Act as a University of Texas at Dallas(sometime refered as UT Dallas or UTD) Guidance Counselor, answering the following questions if they are related to UT Dallas as best you can. You should not use the same action and input twice when using tools. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}

"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [7]:
llm_chain = LLMChain(llm=vertex, prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [8]:
agent_chain.run(input="Compare Hist 1302 and Hist 1301")




> Entering new  chain...
Thought: I need to find the course numbers for Hist 1302 and Hist 1301
Action: Course Search
Action Input: Hist 1302
Observation: found these results:Item 1 with "U.S. History Survey from Civil War - UT Dallas 2017 Undergraduate ..." can be access at "https://catalog.utdallas.edu/2017/undergraduate/courses/hist1302", with snippet "HIST1302 - U.S. History Survey from Civil War. HIST 1302 (HIST 1302) U.S. History Survey from Civil War (3 semester credit hours) An introduction to the ...".Item 2 with "U.S. History Survey from Civil War - UT Dallas 2018 Graduate Catalog" can be access at "https://catalog.utdallas.edu/2018/graduate/courses/hist1302", with snippet "HIST1302 - U.S. History Survey from Civil War. HIST 1302 (HIST 1302) U.S. History Survey from Civil War (3 semester credit hours) An introduction to the ...".Item 3 with "U.S. History Survey from Civil War - UT Dallas 2018 Undergraduate ..." can be access at "https://catalog.utdallas.edu/2018/undergradua

'Hist 1302 is a course that covers U.S. History from the Civil War to the present. Hist 1301 is a course that covers U.S. History from the beginning to the Civil War.'

In [9]:
print(agent_chain.memory.buffer)

Human: Compare Hist 1302 and Hist 1301
AI: Hist 1302 is a course that covers U.S. History from the Civil War to the present. Hist 1301 is a course that covers U.S. History from the beginning to the Civil War.
